<style>
*
{
	text-align: justify;
	line-height: 1.5;
	font-family: "Arial", sans-serif;
	font-size: 12px;
}

h2, h3, h4, h5, h6
{
	font-family: "Arial", sans-serif;
	font-size: 12px;
	font-weight: bold;
}
h2
{
	font-size: 14px;
}
h1
{
	font-family: "Wingdings", sans-serif;
	font-size: 16px;
}
</style>

## Milk Production Prediction Model Builder

<!--
import data_analytics.github as github
print(github.create_jupyter_notebook_header("tahirawwad", "agriculture-data-analytics", "notebooks/notebook-5-03-ml-milk-production.ipynb", "master"))
-->
<table style="margin: auto;"><tr><td><a href="https://mybinder.org/v2/gh/tahirawwad/agriculture-data-analytics/master?filepath=notebooks/notebook-5-03-ml-milk-production.ipynb" target="_parent"><img src="https://mybinder.org/badge_logo.svg" alt="Open In Binder"/></a></td><td>online editors</td><td><a href="https://colab.research.google.com/github/tahirawwad/agriculture-data-analytics/blob/master/notebooks/notebook-5-03-ml-milk-production.ipynbynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></td></tr></table>

### Objective
Create the best machine learning model to predict Milk production value in Ireland according to the historical data from Central Statistics office AEA01 Value at Current Prices for Output, Input and Income in Agriculture.

### Setup

Import required third party Python libraries, import supporting functions and sets up data source file paths.

In [1]:
# Local
#!pip install -r script/requirements.txt
# Remote option
#!pip install -r https://raw.githubusercontent.com/tahirawwad/agriculture-data-analytics/requirements.txt
#Options: --quiet --user

In [2]:
from keras_tuner.tuners import RandomSearch
from pandas import read_csv, DataFrame
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
from xgboost import XGBRegressor
import numpy as np
import os
import pickle
import shutil

### Load dataframe

In [3]:
dataframe_filepath = "./../artifacts/irish-milk-production-eda-output.csv"
dataframe: DataFrame = read_csv(dataframe_filepath)

### Production of Milk

Extract the milk production features.

In [4]:
dataframe = dataframe[[
    'Year', 'All Livestock Products - Milk', 'Taxes on Products',
    'Subsidies on Products', 'Compensation of Employees', 'Contract Work',
    'Entrepreneurial Income', 'Factor Income',
    'Fixed Capital Consumption - Farm Buildings',
    'Fixed Capital Consumption - Machinery, Equipment, etc',
    'Interest less FISIM', 'Operating Surplus', 'Livestock - Cattle',
    'Livestock - Sheep', 'Land Rental',
    'Intermediate Consumption - Contract Work',
    'Intermediate Consumption - Crop Protection Products',
    'Intermediate Consumption - Energy and Lubricants',
    'Intermediate Consumption - Feeding Stuffs',
    'Intermediate Consumption - Fertilisers',
    'Intermediate Consumption - Financial Intermediation Services Indirect',
    'Intermediate Consumption - Forage Plants',
    'Intermediate Consumption - Maintenance and Repairs',
    'Intermediate Consumption - Seeds', 'Intermediate Consumption - Services',
    'Intermediate Consumption - Veterinary Expenses',
    'Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)',
    'Intermediate Consumption - Other Goods and Services'
]]

dataframe.set_index('Year', drop=True, inplace=True)

print("Milk production dataset dimensions", dataframe.shape)

dataframe.head()

Milk production dataset dimensions (31, 27)


,All Livestock Products - Milk,Taxes on Products,Subsidies on Products,Compensation of Employees,Contract Work,Entrepreneurial Income,Factor Income,Fixed Capital Consumption - Farm Buildings,"Fixed Capital Consumption - Machinery, Equipment, etc",Interest less FISIM,...,Intermediate Consumption - Feeding Stuffs,Intermediate Consumption - Fertilisers,Intermediate Consumption - Financial Intermediation Services Indirect,Intermediate Consumption - Forage Plants,Intermediate Consumption - Maintenance and Repairs,Intermediate Consumption - Seeds,Intermediate Consumption - Services,Intermediate Consumption - Veterinary Expenses,"Intermediate Consumption - Other Goods (Detergents, Small Tools, etc)",Intermediate Consumption - Other Goods and Services
Year,,,,,,,,,,,,,,,,,,,,,
1990,1316.3,75.0,408.9,377.6,180.7,1577.2,2321.1,106.4,325.5,267.4,...,642.2,326.0,34.0,553.1,203.9,42.0,109.1,85.2,73.2,182.3
1991,1258.9,78.2,357.3,362.5,172.0,1440.2,2136.5,113.2,331.7,237.2,...,646.5,319.1,39.0,525.6,184.6,43.8,110.2,91.9,75.3,185.5
1992,1373.1,79.5,446.0,336.8,179.8,1842.6,2516.2,117.8,332.4,237.6,...,668.5,305.1,43.0,533.8,175.1,46.4,110.8,93.7,72.2,183.0
1993,1439.0,68.0,466.4,339.0,199.5,1985.6,2586.9,121.9,335.5,162.5,...,733.5,301.2,49.0,572.9,195.7,46.9,113.3,100.2,79.8,193.1
1994,1446.2,53.7,666.0,345.2,205.3,2051.7,2623.7,125.0,346.5,119.3,...,818.6,316.3,56.0,604.6,220.3,50.0,118.5,108.7,81.8,200.3


### Prepare Model Data

#### Select the feature set and the target

Select the feature set and the target 

In [5]:
X = dataframe.iloc[:, 1:].values
Y = dataframe.iloc[:, 0].values.reshape(-1, 1)
print('Features dimension:', np.shape(X))
print('Target dimension:', np.shape(Y))

Features dimension: (31, 26)
Target dimension: (31, 1)


#### Fill missing values

In [6]:
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp_mean.fit_transform(X)
Y = imp_mean.fit_transform(Y)

#### Define Training set 80% Test set 20% 

In [7]:
X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=2021)
print()
print('x_train dimension', X_train.shape)
print('y_train dimension', Y_train.shape)
print()
print('x_test dimension', X_test.shape)
print('y_test dimension', Y_test.shape)


x_train dimension (24, 26)
y_train dimension (24, 1)

x_test dimension (7, 26)
y_test dimension (7, 1)


#### Scale & Transform

In [8]:
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()

scaler_x.fit(X_train)
xtrain_scale = scaler_x.transform(X_train)

scaler_y.fit(Y_train)
ytrain_scale = scaler_y.transform(Y_train)

scaler_x.fit(X_test)
xtest_scale = scaler_x.transform(X_test)

scaler_x.fit(Y_test)
ytest_scale = scaler_y.transform(Y_test)

### Model Testing

#### Score Board

In [9]:
model_scores_dataframe = DataFrame(
    columns=['Model', 'Mean Absolute Error Score(%)'])

#### RandomForest Regressor

##### Train RandomForest

In [10]:
# define Random Forest Regressor
rf_regressor_milk = RandomForestRegressor(random_state=2021)

# define list of Parameters
params_rf_milk = {
    'n_estimators': [100, 500, 800],
    'criterion': ['squared_error', 'absolute_error', 'poisson'],
    'max_features': ["auto", "sqrt", "log2"],
    "bootstrap": [True, False]
}

# Hyper parameter tuning via Grid Search Cross Validation
grid_rf_milk = GridSearchCV(estimator=rf_regressor_milk,
                            param_grid=params_rf_milk,
                            n_jobs=-1,
                            cv=5)

# Fit the grid to scaled data
grid_rf_milk.fit(xtrain_scale, ytrain_scale.reshape(-1))

# print best training model & R squared score
print('Best training model ', grid_rf_milk.best_estimator_)
print('Best training model score, coefficient of determination R squared',
      grid_rf_milk.best_score_)

Best training model  RandomForestRegressor(criterion='poisson', max_features='log2',
                      n_estimators=800, random_state=2021)
Best training model score, coefficient of determination R squared 0.897197143180551


##### Predict RandomForest

In [11]:
# Predict Milk Production and un-scale back to original values
y_predict = scaler_y.inverse_transform(
    grid_rf_milk.predict(xtest_scale).reshape(-1, 1))

print('predicted milk production values \n', y_predict)
print('actual milk production values \n', Y_test)

# Calculate Mean Absolute Error
MAE_rf = mean_absolute_error(Y_test, y_predict)
#print(MAE_rf)

predicted milk production values 
 [[1622.61839583]
 [1623.00755208]
 [1737.02422917]
 [1530.43363542]
 [1499.47470833]
 [1596.36222083]
 [2641.510625  ]]
actual milk production values 
 [[1332.5]
 [1106.5]
 [1566.1]
 [1446.2]
 [1408.8]
 [1431.2]
 [1790.8]]


##### Save Model Score

In [12]:
model_scores_dataframe.loc[len(model_scores_dataframe)] = [
    'RandomForest', MAE_rf
]
print(model_scores_dataframe)

          Model  Mean Absolute Error Score(%)
0  RandomForest                    309.761624


#### XGBOOST Regressor

##### Train XGBOOST

In [13]:
# define XGBRegressor
xgb_regressor_milk = XGBRegressor(random_state=2021)

# define parameters space to loop over
params_xgb_milk = {
    'n_estimators': [20, 40, 80, 160, 340, 500],
    'max_depth': [3, 6, 9],
    'gamma': [0.01, 0.1],
    'learning_rate': [0.001, 0.01, 0.1, 1]
}

# Hyper parameter tuning via Grid Search Cross Validation
grid_xgb_milk = GridSearchCV(
    estimator=xgb_regressor_milk,
    param_grid=params_xgb_milk,
    #n_jobs=-1,
    scoring=['r2', 'neg_root_mean_squared_error'],
    refit='r2',
    n_jobs=-1,
    cv=5,
    verbose=4)

# fit grid to training scaled set
grid_xgb_milk.fit(xtrain_scale, ytrain_scale)

# print best training model & R squared score
print('Best training model ', grid_xgb_milk.best_estimator_)
print('Best model Parameters', grid_xgb_milk.best_params_)
print('Best training model score, coefficient of determination R squared',
      grid_xgb_milk.best_score_)

Fitting 5 folds for each of 144 candidates, totalling 720 fits
Best training model  XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0.01, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=160, n_jobs=8,
             num_parallel_tree=1, predictor='auto', random_state=2021,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)
Best model Parameters {'gamma': 0.01, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 160}
Best training model score, coefficient of determination R squared 0.8477032808741709


##### Predict XGBOOST

In [14]:
# Predict Milk Production and un-scale back to original values
y_predict = scaler_y.inverse_transform(
    grid_xgb_milk.predict(xtest_scale).reshape(-1, 1))

print('predicted milk production values \n', y_predict)
print('actual milk production values \n', Y_test)

# Calculate Mean Absolute Error
MAE_xgb = mean_absolute_error(Y_test, y_predict)
#print(MAE_xgb)

predicted milk production values 
 [[1544.6935]
 [1817.2489]
 [1433.4462]
 [1468.3297]
 [1433.4462]
 [1433.4462]
 [2567.2097]]
actual milk production values 
 [[1332.5]
 [1106.5]
 [1566.1]
 [1446.2]
 [1408.8]
 [1431.2]
 [1790.8]]


##### Save Model Score

In [15]:
model_scores_dataframe.loc[len(model_scores_dataframe)] = ['XGBOOST', MAE_xgb]
print(model_scores_dataframe)

          Model  Mean Absolute Error Score(%)
0  RandomForest                    309.761624
1       XGBOOST                    268.718283


#### ANN Artificial Neural Network

##### Training & Keras Parameter Tuning

In [16]:
temp_directory: str = './../temp/ANN-tuner/'


# Define ANN model with Hyper parameter variable
def build_model(hp):
    model = keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 23)):
        model.add(
            layers.Dense(units=hp.Int('units_' + str(i),
                                      min_value=23,
                                      max_value=600,
                                      step=32),
                         activation='relu'))
        model.add(layers.Dense(1, activation='linear'))
        model.compile(optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
                      loss='mean_absolute_error',
                      metrics=['mean_absolute_error'])
        return model


if os.path.isdir(temp_directory):
    try:
        shutil.rmtree(temp_directory)
    except OSError as exception:
        print(f"Error: {exception.filename} - {exception.strerror}.")

# create a directory to store each iteration of modelling
tuner = RandomSearch(build_model,
                     objective='val_mean_absolute_error',
                     max_trials=5,
                     executions_per_trial=3,
                     directory=temp_directory,
                     project_name='Milk production')

# Defined parameter space to search in
tuner.search_space_summary()

# train trial models and compare with validation set
tuner.search(xtrain_scale,
             ytrain_scale,
             epochs=50,
             validation_data=(xtest_scale, ytest_scale))

# print best 10 models according to val_mean_absolute_error
print('\n')
tuner.results_summary()

# get best model from training trials
bestANNModel = tuner.get_best_models(num_models=1)[0]

# fit best model to training scaled data and scaled test data
bestANNModel.fit(xtrain_scale,
                 ytrain_scale,
                 epochs=50,
                 validation_data=(xtest_scale, ytest_scale))
#Clean up
if os.path.isdir(temp_directory):
    try:
        shutil.rmtree(temp_directory)
    except OSError as exception:
        print(f"Error: {exception.filename} - {exception.strerror}.")

Trial 5 Complete [00h 00m 10s]
val_mean_absolute_error: 0.09046632548173268

Best val_mean_absolute_error So Far: 0.09046632548173268
Total elapsed time: 00h 00m 55s
INFO:tensorflow:Oracle triggered exit


Results summary
Results in ./../temp/ANN-tuner/Milk production
Showing 10 best trials
Objective(name='val_mean_absolute_error', direction='min')
Trial summary
Hyperparameters:
num_layers: 8
units_0: 503
learning_rate: 0.0001
Score: 0.09046632548173268
Trial summary
Hyperparameters:
num_layers: 21
units_0: 375
learning_rate: 0.01
Score: 0.09544547647237778
Trial summary
Hyperparameters:
num_layers: 12
units_0: 279
learning_rate: 0.0001
Score: 0.10072392970323563
Trial summary
Hyperparameters:
num_layers: 13
units_0: 183
learning_rate: 0.001
Score: 0.12763111790021262
Trial summary
Hyperparameters:
num_layers: 19
units_0: 151
learning_rate: 0.0001
Score: 0.16741826881965002
Epoch 1/50
1/1 [==============================] - 1s 731ms/step - loss: 0.1692 - mean_absolute_error: 0.1692 - va

1/1 [==============================] - 0s 34ms/step - loss: 0.0401 - mean_absolute_error: 0.0401 - val_loss: 0.1939 - val_mean_absolute_error: 0.1939
Epoch 47/50
1/1 [==============================] - 0s 46ms/step - loss: 0.0397 - mean_absolute_error: 0.0397 - val_loss: 0.1939 - val_mean_absolute_error: 0.1939
Epoch 48/50
1/1 [==============================] - 0s 44ms/step - loss: 0.0392 - mean_absolute_error: 0.0392 - val_loss: 0.1933 - val_mean_absolute_error: 0.1933
Epoch 49/50
1/1 [==============================] - 0s 44ms/step - loss: 0.0388 - mean_absolute_error: 0.0388 - val_loss: 0.1922 - val_mean_absolute_error: 0.1922
Epoch 50/50
1/1 [==============================] - 0s 53ms/step - loss: 0.0383 - mean_absolute_error: 0.0383 - val_loss: 0.1907 - val_mean_absolute_error: 0.1907


##### Predict

In [17]:
# Predict Milk Production and un-scale back to original values

y_predict = scaler_y.inverse_transform(
    bestANNModel.predict(xtest_scale).reshape(-1, 1))

print('predicted milk production values \n', y_predict)
print('actual milk production values \n', Y_test)

# Calculate Mean Absolute Error
MAE_ANN = mean_absolute_error(Y_test, y_predict)
#print(MAE_xgb)

predicted milk production values 
 [[1709.7432]
 [1529.5222]
 [1547.8958]
 [1567.8647]
 [1487.0751]
 [1512.3175]
 [2685.2715]]
actual milk production values 
 [[1332.5]
 [1106.5]
 [1566.1]
 [1446.2]
 [1408.8]
 [1431.2]
 [1790.8]]


##### Save Model Score

In [18]:
model_scores_dataframe.loc[len(model_scores_dataframe)] = ['ANN', MAE_ANN]
print(model_scores_dataframe)

          Model  Mean Absolute Error Score(%)
0  RandomForest                    309.761624
1       XGBOOST                    268.718283
2           ANN                    284.856920


### Save Artifacts

Save trained model into binary pickle file to use the model later with new input data from web app

In [19]:
READ_BINARY = "rb"
WRITE_BINARY = "wb"

In [20]:
model_directory: str = "milk-production-models"
directory: str = f'./../artifacts/{model_directory}/'

#### Save Scalers

In [21]:
filename: str = 'milk-features-scaler.pickle'
features_scaler_filepath: str = f'{directory}{filename}'

with open(features_scaler_filepath, WRITE_BINARY) as file:
    pickle.dump(scaler_x, file)

filename: str = 'milk-target-scaler.pickle'
target_scaler_filepath: str = f'{directory}{filename}'

with open(target_scaler_filepath, WRITE_BINARY) as file:
    pickle.dump(scaler_y, file)

#### Save Models

In [22]:
filename: str = 'milk-ann-model.h5'
ann_filepath: str = f'{directory}{filename}'
bestANNModel.save(ann_filepath, save_format='h5')

```py
# fit and transform in one line
scaler_x.fit_transform(X_train)
# remeber to inverse the scaling on model output
scaler_x.inverse_transform(xtest_scale)

# Read pickle file into variable to use scaler
scaler_x_pkl_ann = pickle.load(open(f'{directory}feature-scaler.pickle', 'rb'))

with open(target_scaler_filepath, READ_BINARY) as file:
    pickle.dump(scaler_y, file)
# Read pickle file into variable to use scaler

with open(target_scaler_filepath, READ_BINARY) as file:
    target_scaler = pickle.load(file)

print(type(target_scaler))

with open(ann_filepath, WRITE_BINARY) as file:
    pickle.dump(bestANNModel, file)
    
pickle.dump(bestANNModel, open(f'{directory}pkl_ann_milk', 'wb'))

# Read pickle file into variable to use model
model_pkl_ann = pickle.load(open(f'{directory}pkl_ann_milk', 'rb'))

## Example using pickle file with saved ANN model

# take input from source as array
data_input_from_webapp = np.array([357.3, 362.5, 172., 1440.2, 2136.5])

# scale input with same scaler as used in model
scale_data_from_webapp = scaler_x.transform(
    data_input_from_webapp.reshape(1, -1))

# predict scaled value
scaled_prediction = bestANNModel.predict(scale_data_from_webapp)

# descale prediction back to normal value
prediction = scaler_y.inverse_transform(scaled_prediction)
print('\n Expected Milk Production is ', prediction[0][0])
```